# Set Up

In [1]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Install libraries

In [15]:
# Install HuggingFace transformers
%pip install transformers evaluate peft trl bitsandbytes rouge_score

# Install LLM as a Judge
#%pip install prometheus-eval triton vllm


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 32.3 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=4b1180cdb5059d86955e770d48b27a4bdfe0d16d1471424732fb19cff9a0d915
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, logging
from peft import LoraConfig
from trl import SFTTrainer
from transformers import Trainer

import pandas as pd
import numpy as np
import torch
from transformers import set_seed
from transformers import Trainer
from transformers import TrainingArguments
import random
from sklearn.model_selection import train_test_split
import evaluate
import re
from datasets import Dataset
from torch.utils.data import DataLoader, TensorDataset


# # LLM as Eval Judge
# from prometheus_eval.vllm import VLLM
# from prometheus_eval import PrometheusEval
# from prometheus_eval.prompts import ABSOLUTE_PROMPT, SCORE_RUBRIC_TEMPLATE


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [12]:
# Disable WANDB logging
import os

os.environ["WANDB_DISABLED"] = "true"
# # To avoid GPU running out of memory, reduce memory fragmentation
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
base_path = './drive/MyDrive/EndOfStory/'

Output_DIR = base_path + '/results_A100_run6/'

# Set Random Seed


In [17]:
# Set seeds for reproducibility
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed_value)
set_seed(seed_value)

# Datasets

In [18]:
# Datasets file paths, may change per envrionments like local vs Google Colab

ROCStoriesSpring2016_FILE_PATH = base_path +'ROCStoriesSpring2016.csv'
ROCStoriesWinter2017_FILE_PATH = base_path +'ROCStoriesWinter2017.csv'
ClozeTestWinter2018ValData_FILE_PATH = base_path +'ClozeTestWinter2018ValData.csv'
ClozeTestWinter2018TestData_FILE_PATH = base_path +'ClozeTestWinter2018TestData.csv'



In [19]:
# Load data
ROCStoriesSpring2016 = pd.read_csv(ROCStoriesSpring2016_FILE_PATH)
ROCStoriesWinter2017 = pd.read_csv(ROCStoriesWinter2017_FILE_PATH)
ClozeTestWinter2018ValData = pd.read_csv(ClozeTestWinter2018ValData_FILE_PATH)
ClozeTestWinter2018TestData = pd.read_csv(ClozeTestWinter2018TestData_FILE_PATH)


In [20]:
print(ROCStoriesSpring2016.shape)
ROCStoriesSpring2016.head()

(45496, 7)


,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5
0,9a51198e-96f1-42c3-b09d-a3e1e067d803,Overweight Kid,Dan's parents were overweight.,Dan was overweight as well.,The doctors told his parents it was unhealthy.,His parents understood and decided to make a c...,They got themselves and Dan on a diet.
1,617e7ada-3878-488d-bd56-40695b91f053,The Bike Accident,Carrie had just learned how to ride a bike.,She didn't have a bike of her own.,Carrie would sneak rides on her sister's bike.,She got nervous on a hill and crashed into a w...,The bike frame bent and Carrie got a deep gash...
2,79b0da1f-e460-4173-ba58-8c9e2553c53a,Beach,Morgan enjoyed long walks on the beach.,She and her boyfriend decided to go for a long...,"After walking for over a mile, something happe...",Morgan decided to propose to her boyfriend.,Her boyfriend was upset he didn't propose to h...
3,d173b7de-4611-4cdf-934c-912834755e41,The bad customer.,Jane was working at a diner.,"Suddenly, a customer barged up to the counter.",He began yelling about how long his food was t...,Jane didn't know how to react.,"Luckily, her coworker intervened and calmed th..."
4,af0fd5a4-de36-47ba-8aa2-e99d10986d7a,Being Patient,I was talking to my crush today.,She continued to complain about guys flirting ...,I decided to agree with what she says and list...,"After I got home, I got a text from her.",She asked if we can hang out tomorrow.


In [21]:
print(ROCStoriesWinter2017.shape)
ROCStoriesWinter2017.head()

(52665, 7)


,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight re...,He examined his habits to try and figure out t...,He realized he'd been eating too much fast foo...,He stopped going to burger places and started ...,"After a few weeks, he started to feel much bet..."
1,0beabab2-fb49-460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about ...
2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual e...,All of his clothes were either too formal or t...,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for...
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Different Opinions,Bobby thought Bill should buy a trailer and ha...,Bill thought a truck would be better for what ...,Bobby pointed out two vehicles were much more ...,Bill was set in his ways with conventional thi...,He ended up buying the truck he wanted despite...
4,c71bb23b-7731-4233-8298-76ba6886cee1,Overcoming shortcomings,John was a pastor with a very bad memory.,He tried to memorize his sermons many days in ...,He decided to learn to sing to overcome his ha...,He then made all his sermons into music and sa...,His congregation was delighted and so was he.


In [22]:
print(ClozeTestWinter2018ValData.shape)
ClozeTestWinter2018ValData.head()

(1571, 8)


,InputStoryid,InputSentence1,InputSentence2,InputSentence3,InputSentence4,RandomFifthSentenceQuiz1,RandomFifthSentenceQuiz2,AnswerRightEnding
0,138d5bfb-05cc-41e3-bf2c-fa85ebad14e2,Rick grew up in a troubled household.,"He never found good support in family, and tur...",It wasn't long before Rick got shot in a robbery.,The incident caused him to turn a new leaf.,He is happy now.,He joined a gang.,1
1,bff9f820-9605-4875-b9af-fe6f14d04256,Laverne needs to prepare something for her fri...,She decides to bake a batch of brownies.,She chooses a recipe and follows it closely.,Laverne tests one of the brownies to make sure...,The brownies are so delicious Laverne eats two...,Laverne doesn't go to her friend's party.,1
2,e8f628d5-9f97-40ed-8611-fc0e774673c4,Sarah had been dreaming of visiting Europe for...,She had finally saved enough for the trip.,She landed in Spain and traveled east across t...,She didn't like how different everything was.,Sarah then decided to move to Europe.,Sarah decided that she preferred her home over...,2
3,f5226bfe-9f26-4377-b05f-3d9568dbdec1,Gina was worried the cookie dough in the tube ...,She was very happy to find she was wrong.,The cookies from the tube were as good as from...,Gina intended to only eat 2 cookies and save t...,Gina liked the cookies so much she ate them al...,Gina gave the cookies away at her church.,1
4,69ac9b05-b956-402f-9fff-1f926ef9176b,It was my final performance in marching band.,I was playing the snare drum in the band.,We played Thriller and Radar Love.,The performance was flawless.,I was very proud of my performance.,I was very ashamed of my performance.,1


In [23]:
print(ClozeTestWinter2018TestData.shape)
ClozeTestWinter2018TestData.head()

(1571, 7)


,InputStoryid,InputSentence1,InputSentence2,InputSentence3,InputSentence4,RandomFifthSentenceQuiz1,RandomFifthSentenceQuiz2
0,f6aad64a-e34c-415d-b895-dbfa187ed43e,Bob was bored at his job as a school teacher.,He had been working so hard this past month.,He decided to treat himself with something spe...,He ordered tickets for a weekend snowboarding ...,He was looking forward to getting away.,His boss told him he had to work this weekend.
1,0fedd90d-5295-4b79-b2d0-15a2bad624ee,Olivia went out with Harry on a date.,Harry thought the date was going well.,Olivia thinks he is a complete jerk and never ...,Harry keeps raving about their chemistry.,Olivia is about to leave.,"Olivia had her friend call her, to tell her th..."
2,018152fd-f984-4d05-ad1e-12f1fb7eceb6,Jack and Ferris always fought for headphones.,One day Jack broke Ferris' headphones while jo...,Ferris was furious at Jack.,Their parents yelled at them.,Jack promised Ferris to buy him new headphones.,Jack promised to take Ferris jogging.
3,feef76df-b75a-4501-9c1a-f8a7b6ee442f,I needed someone to help me move a bed across ...,"I called a couple of friends, but they were busy.","Finally, I called my grandson, who came right ...",He helped me move the bed to the right spot.,I made him an ice cream sundae for his efforts.,"He was tired from moving it, so he took a nap ..."
4,929eaf8b-a175-4460-a885-43be8a89ca62,Hannah had a beautiful cat that she loved very...,"However, she noticed that her cat was getting ...","One day, her cat ran away and never came back.",Hannah was devastated.,She never saw her beautiful cat again.,"The next day, she saw the cat walking down the..."


In [24]:
pd.merge(ROCStoriesSpring2016, ROCStoriesWinter2017, how='inner').empty

True

In [25]:
# In case some rows have different story ids but the same sentences
pd.merge(ROCStoriesSpring2016, ROCStoriesWinter2017, how='inner', on=["sentence1", "sentence2"]).empty

True

We can see that two datasets have no duplicate. We can safely combine them

In [26]:
ROCStories = pd.concat([ROCStoriesSpring2016, ROCStoriesWinter2017])
ROCStories

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5
0,9a51198e-96f1-42c3-b09d-a3e1e067d803,Overweight Kid,Dan's parents were overweight.,Dan was overweight as well.,The doctors told his parents it was unhealthy.,His parents understood and decided to make a c...,They got themselves and Dan on a diet.
1,617e7ada-3878-488d-bd56-40695b91f053,The Bike Accident,Carrie had just learned how to ride a bike.,She didn't have a bike of her own.,Carrie would sneak rides on her sister's bike.,She got nervous on a hill and crashed into a w...,The bike frame bent and Carrie got a deep gash...
2,79b0da1f-e460-4173-ba58-8c9e2553c53a,Beach,Morgan enjoyed long walks on the beach.,She and her boyfriend decided to go for a long...,"After walking for over a mile, something happe...",Morgan decided to propose to her boyfriend.,Her boyfriend was upset he didn't propose to h...
3,d173b7de-4611-4cdf-934c-912834755e41,The bad customer.,Jane was working at a diner.,"Suddenly, a customer barged up to the counter.",He began yelling about how long his food was t...,Jane didn't know how to react.,"Luckily, her coworker intervened and calmed th..."
4,af0fd5a4-de36-47ba-8aa2-e99d10986d7a,Being Patient,I was talking to my crush today.,She continued to complain about guys flirting ...,I decided to agree with what she says and list...,"After I got home, I got a text from her.",She asked if we can hang out tomorrow.
...,...,...,...,...,...,...,...
52660,134e8636-3617-43d8-ba6a-9a11b3b115b1,Flavor,The man liked the flavor.,He tried to recreate it at home.,He could not get the flavor right.,He asked the owner of the recipe for help.,The owner of the flavor sold him the recipe.
52661,4c317f76-ca42-4024-a4c2-12ec911cf89b,After Death,"After my friend's dad's funeral, I got in trou...",The principal said I wasn't allowed to leave s...,He found out I had my friend sign me out.,He told me I was getting detention.,I skipped detention all week.
52662,a18fd0d2-4d0c-4316-befe-e3d827fe699b,Janice breaks her wrist,Janice was out exercising for her big soccer g...,She was doing some drills with her legs.,While working out and exercising she slips on ...,She falls down and uses her wrist to break her...,She breaks her wrist in the process and goes t...
52663,2c14252b-4080-4fca-8765-537772018508,Jamie marries for love,Jamie is an american girl.,Jamie wants to get married to a mexican man.,Her family assumes it's because the man wants ...,Jamie insist that she is marrying him out of l...,Jamie gets married and they spent the rest of ...


# Utility Functions

In [27]:
def combine_sentences(row, sentence_columns_to_be_combined=["sentence1", "sentence2", "sentence3", "sentence4"]):
  assert len(sentence_columns_to_be_combined) > 0, "Sentence columns to be combined list length has to be larger than 0"
  combined_sentence = row[sentence_columns_to_be_combined[0]]
  for i in range(1, len(sentence_columns_to_be_combined)):
    combined_sentence += ' ' + row[sentence_columns_to_be_combined[i]]
  return combined_sentence


def combine_selected_columns(row, columns=["sentence1", "sentence2", "sentence3", "sentence4"]):
    return ' '.join(str(row[col]) for col in columns if not pd.isna(row[col]))



In [28]:
def remove_first_four_sentences(paragraph):
    # Split the paragraph into sentences using regex
    sentences = re.split(r'(?<=[.!?])[\s"]+', paragraph)
    # Keep all sentences except the first 4
    remaining_sentences = sentences[4:]
    # Rejoin the sentences into a paragraph
    joined_sentences = ""
    for sentence in remaining_sentences:
      joined_sentences += sentence
      joined_sentences += " "
    joined_sentences = joined_sentences[:-1]
    return joined_sentences

def vectorized_remove_first_four_sentences(paragraphs):
    return np.vectorize(remove_first_four_sentences)(np.array(paragraphs))

def concat_story_body_with_endings(story_body, story_ending):
  return [f"{p1} {p2}" for p1, p2 in zip(story_body, story_ending)]

In [29]:
# def apply_prompt_template(input_text):
#   instruction = "Complete the story in a single sentence, based on the story beginning provided."
#   prompt_input = f"Below is an instruction that describes a task, paired with an input that provides further context. " \
#         "Write a response that appropriately completes the request.\n\n" \
#         f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response: "
#   return prompt_input

def apply_prompt_template(input_text):
  instruction = "Complete the story using a single sentence, based on the story beginning provided."
  prompt_input = f"Below is an instruction that describes a task, paired with an input that provides further context. " \
        "Write a response that appropriately completes the request.\n\n" \
        f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response: "
  return prompt_input

def remove_newline(input_string):
  #cleaned_string = re.sub(r'[^\w\s]', '', input_string)  # Remove special characters
  cleaned_string = input_string.replace('\n', '').replace('\r', '').replace('<br>','')  # Remove newline characters
  return cleaned_string

def remove_prompt_from_generated_text(input):
  input = remove_newline(input)
  #print(input)
  story_context_start_idx = input.find('### Input:')
  story_context_end_idx = input.find('### Response:')

  out = ""
  out += input[story_context_start_idx + 10: story_context_end_idx]
  out += input[story_context_end_idx + 13:]

  return out

def extract_story_beginning_from_generated_text(input):
  input = remove_newline(input)

  story_context_start_idx = input.find('### Input:')
  story_context_end_idx = input.find('### Response:')

  out = ""
  out += input[story_context_start_idx + 10: story_context_end_idx]

  return out


def extract_story_ending_from_generated_text(input):
  input = remove_newline(input)

  response_start_idx = input.find('### Response:')

  out = ""
  out += input[response_start_idx + 13: ]
  return remove_newline(out)

def vectorized_extract_story_beginning_from_generated_text(paragraphs):
    return np.vectorize(extract_story_beginning_from_generated_text)(np.array(paragraphs))

def vectorized_extract_story_ending_from_generated_text(paragraphs):
    return np.vectorize(extract_story_ending_from_generated_text)(np.array(paragraphs))


def vectorized_remove_prompt_from_generated_text(paragraphs):
    return np.vectorize(remove_prompt_from_generated_text)(np.array(paragraphs))

# Train, Val & Test Split

In [30]:
ROCStories_Y = ROCStories["sentence5"]
ROCStories_X = ROCStories.drop(columns=["storyid", "storytitle", "sentence5"])

ROCStories_X = ROCStories_X.apply(combine_selected_columns, axis=1)



In [31]:
ROCStories_X = ROCStories_X.apply(apply_prompt_template)

ROCStories_X.iloc[0]

"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nComplete the story using a single sentence, based on the story beginning provided.\n\n### Input:\nDan's parents were overweight. Dan was overweight as well. The doctors told his parents it was unhealthy. His parents understood and decided to make a change.\n\n### Response: "

In [32]:
ROCStories_X.shape

(98161,)

In [33]:
ROCStories_X.shape

(98161,)

In [34]:
ROCStories_Y

,sentence5
0,They got themselves and Dan on a diet.
1,The bike frame bent and Carrie got a deep gash...
2,Her boyfriend was upset he didn't propose to h...
3,"Luckily, her coworker intervened and calmed th..."
4,She asked if we can hang out tomorrow.
...,...
52660,The owner of the flavor sold him the recipe.
52661,I skipped detention all week.
52662,She breaks her wrist in the process and goes t...
52663,Jamie gets married and they spent the rest of ...


In [35]:
ROCStories_Y.shape

(98161,)

We decided to combine some of ROCStories data and the Story Cloze 2018 Validation datasets as the final train datasets. The reasoning to additionally include Story Cloze dataset is that Story Cloze 2018 Validation datasets have the correct endings, therefore can be used for both classification or story ending generation evaluation. It also has a potential to be a bit different from the original ROCStories datasets that is mainly used as training data (in other words, increasing the variance of the test datasets for generalization).

In [36]:
ROCX_train_val, ROCX_test, ROCy_train_val, ROCy_test = train_test_split(ROCStories_X, ROCStories_Y, test_size=0.02, random_state=seed_value)

In [37]:
print(ROCX_train_val.shape)
print(ROCX_test.shape)
print(ROCy_train_val.shape)
print(ROCy_test.shape)

(96197,)
(1964,)
(96197,)
(1964,)


In [38]:
ClozeX_test = ClozeTestWinter2018ValData.drop(columns=["InputStoryid", "RandomFifthSentenceQuiz1", "RandomFifthSentenceQuiz2", "AnswerRightEnding"])
ClozeX_test = ClozeX_test.rename(columns={"InputSentence1": "sentence1", "InputSentence2": "sentence2", "InputSentence3": "sentence3", "InputSentence4": "sentence4"})
ClozeX_test = ClozeX_test.apply(combine_selected_columns, axis=1)
ClozeX_test = ClozeX_test.apply(apply_prompt_template)

ClozeX_test

,0
0,"Below is an instruction that describes a task,..."
1,"Below is an instruction that describes a task,..."
2,"Below is an instruction that describes a task,..."
3,"Below is an instruction that describes a task,..."
4,"Below is an instruction that describes a task,..."
...,...
1566,"Below is an instruction that describes a task,..."
1567,"Below is an instruction that describes a task,..."
1568,"Below is an instruction that describes a task,..."
1569,"Below is an instruction that describes a task,..."


In [39]:
Clozey_test = np.where(ClozeTestWinter2018ValData["AnswerRightEnding"]==1, ClozeTestWinter2018ValData["RandomFifthSentenceQuiz1"], ClozeTestWinter2018ValData["RandomFifthSentenceQuiz2"])
Clozey_test = pd.DataFrame(Clozey_test, columns=["sentence5"])
Clozey_test

,sentence5
0,He is happy now.
1,The brownies are so delicious Laverne eats two...
2,Sarah decided that she preferred her home over...
3,Gina liked the cookies so much she ate them al...
4,I was very proud of my performance.
...,...
1566,I have very fond memories of checkers.
1567,She loved her new phone.
1568,They were on sale.
1569,She was offered the new job at a higher salary.


In [40]:
ClozeX_test.iloc[0]

"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nComplete the story using a single sentence, based on the story beginning provided.\n\n### Input:\nRick grew up in a troubled household. He never found good support in family, and turned to gangs. It wasn't long before Rick got shot in a robbery. The incident caused him to turn a new leaf.\n\n### Response: "

In [41]:
X_test = pd.concat([ROCX_test, ClozeX_test])
X_test

,0
20391,"Below is an instruction that describes a task,..."
32389,"Below is an instruction that describes a task,..."
8700,"Below is an instruction that describes a task,..."
45048,"Below is an instruction that describes a task,..."
23146,"Below is an instruction that describes a task,..."
...,...
1566,"Below is an instruction that describes a task,..."
1567,"Below is an instruction that describes a task,..."
1568,"Below is an instruction that describes a task,..."
1569,"Below is an instruction that describes a task,..."


In [42]:
y_test = pd.concat([ROCy_test, Clozey_test])
y_test

,sentence5
20391,Evan knew he looked cool in the new car.
32389,Together they were able to surprise him perfec...
8700,Fred apologized to the man.
45048,Talking with him not only made my day it made ...
23146,Tom's friends thought it was annoying.
...,...
1566,I have very fond memories of checkers.
1567,She loved her new phone.
1568,They were on sale.
1569,She was offered the new job at a higher salary.


In [43]:
ROCX_train, ROCX_val, ROCy_train, ROCy_val = train_test_split(ROCX_train_val, ROCy_train_val, test_size=0.2, random_state=seed_value)

In [44]:
ROCX_train = ROCX_train.to_list()
ROCX_val = ROCX_val.to_list()
ROCy_train = ROCy_train.to_list()
ROCy_val = ROCy_val.to_list()
X_test = X_test.to_list()
y_test = y_test["sentence5"].to_list()

In [45]:
print(len(ROCX_train))
print(len(ROCX_val))
print(len(ROCy_train))
print(len(ROCy_val))
print(len(X_test))
print(len(y_test))

76957
19240
76957
19240
3535
3535


In [46]:
X_test[0]

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nComplete the story using a single sentence, based on the story beginning provided.\n\n### Input:\nEvan had been saving for years. He went to the dealership and bought a really fancy BMW. Evan was so proud of his new car. He showed it off around town.\n\n### Response: '

# LLaMA Model and Baseline

TinyLlama 1.1B model from  https://huggingface.co/TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T

In [ ]:
base_model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
# the model will be saved with this name
new_model_name = "llama-1.1B-rocstories_lora_6"

In [ ]:
# Load the model
model = AutoModelForCausalLM.from_pretrained(base_model_name)
model.config.use_cache = False
model.config.pretraining_tp = 1 # parallelism = false cuz we have only 1 device.


In [ ]:
model.to("cuda")

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): 

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token #pad sequences
tokenizer.padding_side = 'right'

In [ ]:
X_test[0]

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nComplete the story in a single sentence, based on the story beginning provided.\n\n### Input:\nEvan had been saving for years. He went to the dealership and bought a really fancy BMW. Evan was so proud of his new car. He showed it off around town.\n\n### Response: '

Generating story endings with baseline model

In [ ]:
print("=====Start of results with Baseline model")

for i in range(5):
  print("Story", i)
  # Use the first row of dataset as a toy example
  story_beginning = X_test[i]

  device = "cuda" if torch.cuda.is_available() else "cpu"

  ## BOTH model and inputs need to be on the same device - CUDA or CPU otherwise there will be errors

  # Load the model and tokenizer, and move the model to the device
  pipe = pipeline(task="text-generation", model=model.to(device), tokenizer=tokenizer)

  # Prepare the input and move it to the correct device
  inputs = tokenizer(story_beginning, return_tensors="pt").input_ids.to(device)

  # Generate the story ending using beam search
  result = model.generate(
      inputs,
      max_new_tokens=25,                # Limiting to one sentence
      no_repeat_ngram_size=3,           # To avoid repetition
      early_stopping=True           # End generation when all beams finish
  )

  # Decode the generated ending
  generated_text = tokenizer.decode(result[0], skip_special_tokens=True)

  # Print the generated conclusion
  print(generated_text)
  print("Reference:",y_test[i])
  print("-----------\n")

print("=====End of results with Baseline model")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


=====Start of results with Baseline model
Story 0


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Complete the story using a single sentence, based on the story beginning provided.

### Input:
Evan had been saving for years. He went to the dealership and bought a really fancy BMW. Evan was so proud of his new car. He showed it off around town.

### Response: 
Even though Evan had saved for years, he still had to pay for the car. Evean had to
Reference: Evan knew he looked cool in the new car.
-----------

Story 1


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Complete the story using a single sentence, based on the story beginning provided.

### Input:
Serena was planning a surprise for her husband's birthday. She wanted to throw him a party, but his schedule was tough. He would always arrive home at widely different times. To get around it, she worked with his co-workers.

### Response: 

```
Serenna was planning to throw a surprise party for her husbands birthday, but he was
Reference: Together they were able to surprise him perfectly.
-----------

Story 2


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Complete the story using a single sentence, based on the story beginning provided.

### Input:
Fred slapped another man's butt. He thought he knew the man. The man turned around. It was not the man he knew.

### Response: 
FRED SLAPPED ANOTHER MAN'S BUTT. HE THOUGHT HE K
Reference: Fred apologized to the man.
-----------

Story 3


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Complete the story using a single sentence, based on the story beginning provided.

### Input:
I used to lived in Phoenix Arizona. On my way home from work, i met my favorite actor. I was shocked by surprise when he actually spoke to me. We had lunch at the Olive Garden.

### Response: 
I lived in Arizona. I met my actor friend at the airport. We went to lunch and he was very
Reference: Talking with him not only made my day it made my week.
-----------

Story 4
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Complete the story using a single sentence, based on the story beginning provided.

### Input:
Tom thought he was really strong. He challenged all his friends to arm wrestle. Tom was

In [ ]:
y_test[0]

'Evan knew he looked cool in the new car.'

In [ ]:
print(remove_prompt_from_generated_text(generated_text))

Tom thought he was really strong. He challenged all his friends to arm wrestle. Tom was able to beat them all. He would brag about it often. Tom was really good at arm wrestling. He was able beat all his friend's. He could beat them easily


In [ ]:
print(extract_story_ending_from_generated_text(generated_text))

 Tom was really good at arm wrestling. He was able beat all his friend's. He could beat them easily


In [ ]:
# Tokenize the input text
inputs = tokenizer(X_test, padding=True, return_tensors="pt")
inputs

{'input_ids': tensor([[    1, 13866,   338,  ...,     2,     2,     2],
        [    1, 13866,   338,  ...,     2,     2,     2],
        [    1, 13866,   338,  ...,     2,     2,     2],
        ...,
        [    1, 13866,   338,  ...,     2,     2,     2],
        [    1, 13866,   338,  ...,     2,     2,     2],
        [    1, 13866,   338,  ...,     2,     2,     2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
inputs['input_ids'].device

device(type='cpu')

Both the model and the inputs need to be on GPU.

In [ ]:
inputs["input_ids"] = inputs["input_ids"].to("cuda")
inputs["attention_mask"] = inputs["attention_mask"].to("cuda")

TODO: It's very consuming to do generate. So let's use only a small subset of test data for now.

In [ ]:
%%time
# Generate the story ending
with torch.no_grad():
  generated_ids = model.generate(inputs["input_ids"][:50], max_new_tokens=25)
generated_ids

CPU times: user 1.49 s, sys: 4.69 ms, total: 1.49 s
Wall time: 1.51 s


tensor([[    1, 13866,   338,  ..., 29896, 29929,  4259],
        [    1, 13866,   338,  ..., 29929,  4259,   338],
        [    1, 13866,   338,  ..., 29896, 29929,  4259],
        ...,
        [    1, 13866,   338,  ..., 29896, 29929,  4259],
        [    1, 13866,   338,  ..., 29929,  4259,   338],
        [    1, 13866,   338,  ...,   313, 29940, 13367]], device='cuda:0')

In [ ]:
# Decode the generated ending
# Please note that currently generated text includes both the input, story beginning, and the generated ending
full_story_with_ending_test = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
full_story_with_ending_test[0]

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nComplete the story using a single sentence, based on the story beginning provided.\n\n### Input:\nEvan had been saving for years. He went to the dealership and bought a really fancy BMW. Evan was so proud of his new car. He showed it off around town.\n\n### Response:  The 2018-19 season is in the books for the 2018-19 season'

# Batch processing with TinyLlama Baseline

:Move inputs to CUDA so that model.generate(**inputs) will use GPU. Once you do this, you can see GPU RAM increases and RAM (CPU RAM) is relatively stable. Otherwise, only CPU is used, which is slow, roughly taking 3 to 4 hours for generating ids for 3500ish test data.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

inputs["input_ids"] = inputs["input_ids"].to(device)
inputs["attention_mask"] = inputs["attention_mask"].to(device)

In [ ]:
inputs['input_ids'].device

device(type='cuda', index=0)

In [52]:
batch_size = 32

# Using a batch size of 32 during inferencing actually helps if there are errors
inputs_dataloader = DataLoader(TensorDataset(inputs["input_ids"], inputs["attention_mask"]), batch_size=batch_size)
original_story_with_endings_dataloader = DataLoader(concat_story_body_with_endings(X_test, y_test), batch_size=batch_size)
story_endings_dataloader = DataLoader(y_test, batch_size=batch_size)

NameError: name 'inputs' is not defined

# Perplexity with original stories

In [ ]:
%%time


perplexity = evaluate.load("perplexity", module_type="metric")
final_perplexity_original_story = 0
for reference in original_story_with_endings_dataloader:
  weight_factor = len(reference) / len(X_test)
  final_perplexity_original_story += weight_factor * perplexity.compute(predictions=reference, model_id=base_model_name)["mean_perplexity"];

final_perplexity_original_story

# Perplexity and ROUGE with pre-trained TinyLlama as baseline

In [ ]:
%%time

rouge = evaluate.load('rouge')
perplexity = evaluate.load("perplexity", module_type="metric")

generated_text_baseline = []

final_perplexity_baseline = 0
final_rouge1_baseline = 0
final_rouge2_baseline = 0
final_rougeL_baseline = 0
with torch.no_grad():
  for input_and_attention_mask, story_reference, ending_reference in zip(inputs_dataloader, original_story_with_endings_dataloader, story_endings_dataloader):

    generated_ids = model.generate(inputs=input_and_attention_mask[0],
                                   attention_mask=input_and_attention_mask[1],
                                   max_new_tokens=25,
                                   early_stopping=True
                                  )
    full_story_with_ending_test = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    for text in full_story_with_ending_test:
      generated_text_baseline.append(text)

    # Calculate perplexity using full story, after removing prompt
    weight_factor = len(ending_reference) / len(X_test)
    final_perplexity_baseline += weight_factor * perplexity.compute(predictions=vectorized_remove_prompt_from_generated_text(full_story_with_ending_test), model_id='TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T')["mean_perplexity"]
    # Calculate rouge using only endings
    # Pop empty strings given in some rare cases, when model doesn't copy down the exact prompt, especially missing punctuations
    empty_indices = [i for i, s in enumerate(vectorized_remove_prompt_from_generated_text(full_story_with_ending_test)) if s == ""]
    if len(empty_indices) > 0:
      print(empty_indices)
    # empty_indices.sort(reverse=True)
    # for index in empty_indices:
    #   full_story_with_ending_test.pop(index)
    #   story_reference.pop(index)

    rouge_results = rouge.compute(predictions=vectorized_extract_story_ending_from_generated_text(full_story_with_ending_test), references=ending_reference)
    final_rouge1_baseline += weight_factor * rouge_results["rouge1"]
    final_rouge2_baseline += weight_factor * rouge_results["rouge2"]
    final_rougeL_baseline += weight_factor * rouge_results["rougeL"]



  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 5min 42s, sys: 34.3 s, total: 6min 16s
Wall time: 6min 34s


In [ ]:
# Store stories into text file

if not os.path.exists(Output_DIR):
  os.makedirs(Output_DIR)


filepaths = [os.path.join(Output_DIR, 'generated_stories_beginning_baseline.txt'),
                          os.path.join(Output_DIR, 'generated_stories_ending_baseline.txt'),
            os.path.join(Output_DIR, 'generated_text_baseline.txt')]

print(filepaths)

with open(filepaths[0], 'w') as f_beg , open(filepaths[1], 'w') as f_end, open(filepaths[2], 'w') as f_text:
  for text in generated_text_baseline:
    f_text.write(remove_newline(text) +'\n')
    f_beg.write(extract_story_beginning_from_generated_text(text) +'\n')
    f_end.write(extract_story_ending_from_generated_text(text) +'\n')


['./drive/MyDrive/EndOfStory//results_A100_run6/generated_stories_beginning_baseline.txt', './drive/MyDrive/EndOfStory//results_A100_run6/generated_stories_ending_baseline.txt', './drive/MyDrive/EndOfStory//results_A100_run6/generated_text_baseline.txt']


In [ ]:
print(final_perplexity_baseline)
print(final_rouge1_baseline)
print(final_rouge2_baseline)
print(final_rougeL_baseline)

12.814715036340951
0.17196993372661393
0.024782822437946706
0.14831231254775165


# TinyLlama model LoRA Fine Tuning

In [ ]:
train_encodings = tokenizer(ROCX_train, padding=True)
val_encodings = tokenizer(ROCX_val, padding=True)

In [ ]:
train_labels = tokenizer(ROCy_train, padding=True, max_length=128, return_tensors="pt")['input_ids']
val_labels = tokenizer(ROCy_val, padding=True, max_length=128, return_tensors="pt")['input_ids']

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2852: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [ ]:
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels
})

In [ ]:
from peft import get_peft_model, LoraConfig
from peft import TaskType
from peft import PeftModel


Clearing up GPU memory

In [ ]:
#Garbage collection
import gc
gc.collect()
for _ in range(11):
  torch.cuda.empty_cache()


I would like to thank the maintainers of this repository for helping us find good LoRA parameters. After multiple iterations of failures, we finally have something that gives a good output. https://github.com/AviSoori1x/Tuning-the-Finetuning/blob/main/Step%202%20Fine%20tuning%20using%20%20LoRA.py

In [ ]:

#If only targeting attention blocks of the model
target_modules_only_attention = ["q_proj", "v_proj", "k_proj", "o_proj"]

#If targeting all linear layers
target_modules = ['q_proj','k_proj','v_proj','o_proj','gate_proj','down_proj','up_proj','lm_head']

# LoRA config
lora_config = LoraConfig(lora_alpha=16, #multiplier of lora output when it's added to the full forward output, recommendation is 2* rank
                        lora_dropout = 0.05, #with a probability of 5% it will set random lora outputs to 0
                        r=8, # most important
                        target_modules = target_modules_only_attention,
                        bias="none",
                        task_type="CAUSAL_LM"
)


# Integrate LoRA with the model
model = get_peft_model(model, lora_config)


per_device_train_batch_size = 48
gradient_accumulation_steps = 2
optim = 'adamw_hf'
learning_rate = 1e-4 # earlier it was 5e-4 which is a little high
max_grad_norm = 1.0 #commonly used
warmup_ratio = 0.03
lr_scheduler_type = "cosine"
steps_quanta = 500

# SFT Arguments
training_args = TrainingArguments(output_dir=Output_DIR,
                                  logging_dir="./logs",
                                  overwrite_output_dir=True,
                                    num_train_epochs=3.0,
                                    per_device_train_batch_size=per_device_train_batch_size,
                                    gradient_accumulation_steps=gradient_accumulation_steps,  # total batch size = 32 * 2 = 64
                                    optim=optim,
                                    save_steps=steps_quanta, #checkpoint every X steps
                                    logging_steps=steps_quanta, #log every step
                                    learning_rate=learning_rate, #step size in optimizer update
                                    weight_decay=0.01, #higher regularization
                                    fp16=True, #enable mixed precision
                                    max_grad_norm=max_grad_norm, #gradient clipping to improve convergence,
                                    max_steps=-1, #using epochs, not steps
                                    group_by_length=True,
                                    warmup_ratio=warmup_ratio,
                                    lr_scheduler_type=lr_scheduler_type, #linear scheduler
                                    evaluation_strategy="steps",
                                    save_strategy="steps",
                                    load_best_model_at_end=True,
                                    eval_steps=steps_quanta,  # Evaluate frequently
                                    save_total_limit=1, #keep only 1 checkpoint
                                  )

trainer=SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=lora_config, #LORA
    dataset_text_field="text",
    args=training_args,
    )

#Train the model
trainer.train()



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warn

Step,Training Loss,Validation Loss
500,1.045500,0.885196
1000,0.874400,0.879028
1500,0.868800,0.876151
2000,0.867200,0.875009


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=2406, training_loss=0.9054629864141729, metrics={'train_runtime': 2891.8421, 'train_samples_per_second': 79.835, 'train_steps_per_second': 0.832, 'total_flos': 2.513269655350272e+17, 'train_loss': 0.9054629864141729, 'epoch': 3.0})

Let's check the Peft config we have in our model.

In [ ]:
if isinstance(model, PeftModel):
    print("This model has been fine-tuned using PEFT.")
else:
    print("This model is not fine-tuned.")

This model has been fine-tuned using PEFT.


In [ ]:
model.print_trainable_parameters()

trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044


Test the fine tuned model for a single inference

In [ ]:
lora_model = model

In [ ]:
# from peft import PeftConfig

# new_base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

# # Check Google drive for the correct path and checkpoint folder
# lora_config_file_path = os.path.join(Output_DIR +'checkpoint-3606/')
# print(lora_config_file_path)
# config = PeftConfig.from_pretrained(Output_DIR +'checkpoint-3606/')
# print(config)
# reloaded_model = PeftModel.from_pretrained(new_base_model,
#             lora_config_file_path,
#             lora_config=config,
#             is_trainable=True # 👈 necessary
# )
# tokenizer = AutoTokenizer.from_pretrained(base_model_name)
# reloaded_model.to("cuda")
# # eval mode
# reloaded_model.eval()

Generating text with LoRA Fine tuned model


In [ ]:
print("=====Start of results with LoRA fine tuned model")



for i in range(5):
  print("Story", i)
  # Use the first row of dataset as a toy example
  story_beginning = X_test[i]

  device = "cuda" if torch.cuda.is_available() else "cpu"

  # Prepare the input and move it to the correct device
  inputs = tokenizer(story_beginning, return_tensors="pt").input_ids.to(device)

  # Generate the story ending using beam search
  result = lora_model.generate(
      inputs,
      max_new_tokens=25,                # Limiting to one sentence
  )

  # Decode the generated ending
  generated_text = tokenizer.decode(result[0], skip_special_tokens=True)

  # Print the generated conclusion
  print(generated_text)
  print("Reference:",y_test[i])
  print("-----------\n")

print("=====End of results with LoRA fine tuned model")


=====Start of results with LoRA fine tuned model
Story 0
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Complete the story using a single sentence, based on the story beginning provided.

### Input:
Evan had been saving for years. He went to the dealership and bought a really fancy BMW. Evan was so proud of his new car. He showed it off around town.

### Response: 
Evan was driving home one day. He saw a car in front of him. It was a police car. E
Reference: Evan knew he looked cool in the new car.
-----------

Story 1
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Complete the story using a single sentence, based on the story beginning provided.

### Input:
Serena was planning a surprise for her husband's birthday. She wanted to th

# Perplexity and ROUGE with the LoRA Fine Tuned Model

In [ ]:
%%time

rouge = evaluate.load('rouge')
perplexity = evaluate.load("perplexity", module_type="metric")

generated_text_lora = []
final_perplexity_lora = 0
final_rouge1_lora = 0
final_rouge2_lora = 0
final_rougeL_lora = 0
with torch.no_grad():
  for input_and_attention_mask, story_reference, ending_reference in zip(inputs_dataloader, original_story_with_endings_dataloader, story_endings_dataloader):
    generated_ids = model.generate(inputs=input_and_attention_mask[0],
                                   attention_mask=input_and_attention_mask[1],
                                   max_new_tokens=25,
                                  # num_beams=4,                  # Using beam search with 4 beams
                                  # no_repeat_ngram_size=2,       # To avoid repetition
                                  # early_stopping=True           # End generation when all beams finish
                                  )
    full_story_with_ending_test = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    for text in full_story_with_ending_test:
      generated_text_lora.append(text)
    # Calculate perplexity using full story, after removing prompt
    weight_factor = len(ending_reference) / len(X_test)
    final_perplexity_lora += weight_factor * perplexity.compute(predictions=vectorized_remove_prompt_from_generated_text(full_story_with_ending_test), model_id='TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T')["mean_perplexity"]
    # Calculate rouge using only endings
    # Pop empty strings given in some rare cases, when model doesn't copy down the exact prompt, especially missing punctuations
    empty_indices = [i for i, s in enumerate(vectorized_remove_prompt_from_generated_text(full_story_with_ending_test)) if s == ""]
    empty_indices.sort(reverse=True)
    for index in empty_indices:
      full_story_with_ending_test.pop(index)
      story_reference.pop(index)

    rouge_results = rouge.compute(predictions=vectorized_extract_story_ending_from_generated_text(full_story_with_ending_test), references=ending_reference)
    final_rouge1_lora += weight_factor * rouge_results["rouge1"]
    final_rouge2_lora += weight_factor * rouge_results["rouge2"]
    final_rougeL_lora += weight_factor * rouge_results["rougeL"]


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 6min 13s, sys: 38.6 s, total: 6min 52s
Wall time: 7min 6s


In [ ]:
print(final_perplexity_lora)
print(final_rouge1_lora)
print(final_rouge2_lora)
print(final_rougeL_lora)
print(len(generated_text_lora))

14.495527136072917
0.1895889209571719
0.03555246622416554
0.162905555535987
3535


In [ ]:
# Store stories into text file

if not os.path.exists(Output_DIR):
  os.makedirs(Output_DIR)


filepaths = [os.path.join(Output_DIR, 'generated_stories_beginning_lora_no_beam.txt'),
                          os.path.join(Output_DIR, 'generated_stories_ending_lora_no_beam.txt'),
            os.path.join(Output_DIR, 'generated_text_lora_no_beam.txt')]

print(filepaths)

with open(filepaths[0], 'w') as f_beg , open(filepaths[1], 'w') as f_end, open(filepaths[2], 'w') as f_text:
  for text in generated_text_lora:
    f_text.write(remove_newline(text) +'\n')
    f_beg.write(extract_story_beginning_from_generated_text(text) +'\n')
    f_end.write(extract_story_ending_from_generated_text(text) +'\n')


['./drive/MyDrive/EndOfStory//results_A100_run6/generated_stories_beginning_lora_no_beam.txt', './drive/MyDrive/EndOfStory//results_A100_run6/generated_stories_ending_lora_no_beam.txt', './drive/MyDrive/EndOfStory//results_A100_run6/generated_text_lora_no_beam.txt']


In [ ]:

# To Stop Google Colab GPU after training is done
# from google.colab import runtime

# runtime.unassign()

In [2]:
%pip install prometheus-eval
%pip install triton
%pip install vllm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.6 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.0
    Uninstalling httpx-0.28.0:
      Successfully uninstalled httpx-0.28.0
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.0/87.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.1 

In [3]:
# LLM as Eval Judge
from prometheus_eval.vllm import VLLM
from prometheus_eval import PrometheusEval
from prometheus_eval.prompts import ABSOLUTE_PROMPT, SCORE_RUBRIC_TEMPLATE

In [4]:
judge_model = VLLM(model="prometheus-eval/prometheus-7b-v2.0", gpu_memory_utilization=0.5, enforce_eager=True, max_model_len=2048)

judge = PrometheusEval(model=judge_model, absolute_grade_template=ABSOLUTE_PROMPT)
rubric_data = {
  "criteria":"Does the model successfully generate a response that is a suitable ending to the provided story body, in terms of language fluency, semantics coherence, and story flow",
  "score1_description":"The ending is riddled with language errors, is incoherent or disconnected from the story body, and disrupts the narrative flow. It leaves the reader confused or unsatisfied.",
  "score2_description":"The ending has noticeable language issues, inconsistencies, or rushed transitions. While it ties some loose ends, it feels incomplete or awkward.",
  "score3_description":"The ending is adequately written with minor language errors and mostly logical progression, but it lacks emotional impact or creativity in its resolution.",
  "score4_description":"The ending is well-crafted, fluent, and coherent, with a fitting and satisfying resolution. It enhances the story’s themes and characters, though it might lack exceptional originality or depth.",
  "score5_description":"The ending is flawless in language fluency, beautifully integrates with the story, and delivers a compelling, imaginative, and emotionally resonant conclusion that elevates the entire narrative."
}
score_rubric = SCORE_RUBRIC_TEMPLATE.format(**rubric_data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

INFO 12-08 19:17:13 config.py:350] This model supports multiple tasks: {'generate', 'embedding'}. Defaulting to 'generate'.
WARNING 12-08 19:17:13 config.py:503] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 12-08 19:17:13 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='prometheus-eval/prometheus-7b-v2.0', speculative_config=None, tokenizer='prometheus-eval/prometheus-7b-v2.0', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), ob

tokenizer_config.json:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

...

model-00004-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00006-of-00008.safetensors:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/22.8k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/8 [00:00<?, ?it/s]


INFO 12-08 19:18:08 model_runner.py:1077] Loading model weights took 13.4966 GB
INFO 12-08 19:18:09 worker.py:232] Memory profiling results: total_gpu_memory=39.56GiB initial_memory_usage=14.00GiB peak_torch_memory=13.81GiB memory_usage_post_profile=14.01GiB non_torch_memory=0.51GiB kv_cache_size=5.46GiB gpu_memory_utilization=0.50
INFO 12-08 19:18:10 gpu_executor.py:113] # GPU blocks: 2796, # CPU blocks: 2048
INFO 12-08 19:18:10 gpu_executor.py:117] Maximum concurrency for 2048 tokens per request: 21.84x


In [ ]:
X_test[0]

In [ ]:
original_story_bodies = X_test
original_story_endings_reference = y_test

In [ ]:
X_test_story_only = vectorized_extract_story_beginning_from_generated_text(X_test)

X_test_story_only[0]

In [ ]:
len(X_test_story_only[0])

In [ ]:
len(y_test[0])

In [ ]:
%%time
LLM_judge_score_original_story = 0
for original_stories, reference in zip(original_story_with_endings_dataloader, story_endings_dataloader):
  print(len(original_stories), len(reference))
  feedback, score = judge.absolute_grade(
      instructions=original_stories,
      responses=reference,
      rubric=score_rubric,
      reference_answers=reference
    )
  LLM_judge_score_original_story += sum(score)
LLM_judge_score_original_story = LLM_judge_score_original_story/len(X_test)
LLM_judge_score_original_story

In [ ]:
generated_story_endings = []

for text in generated_text_lora:
  ending = extract_story_ending_from_generated_text(text)
  generated_story_endings.append(ending)

len(generated_story_endings)

In [ ]:
generated_story_endings[0]

In [ ]:
original_story_with_endings_dataloader

In [ ]:
instructions_dataloader = DataLoader(X_test, batch_size=batch_size)
reference_dataloader = DataLoader(y_test, batch_size=batch_size)
generated_endings_dataloader = DataLoader(generated_story_endings, batch_size=batch_size)

In [ ]:
%%time
LLM_judge_score_fine_tuning = 0
i = 0
for instruction, reference, gen_ending in zip(instructions_dataloader, reference_dataloader,generated_endings_dataloader):
  i += 1
  feedback, score = judge.absolute_grade(
      instructions=instruction,
      responses=gen_ending,
      rubric=score_rubric,
      reference_answers=reference
    )
  print("Iteration", i, score)
  LLM_judge_score_fine_tuning += sum(score)
LLM_judge_score_fine_tuning = LLM_judge_score_fine_tuning/len(X_test)
LLM_judge_score_fine_tuning

# LLM as a judge eval on Fine tuned model

In [ ]:
generated_story_endings = []

for text in generated_text_lora:
  ending = extract_story_ending_from_generated_text(text)
  generated_story_endings.append(ending)

len(generated_story_endings)

After removing extra params (beam search, early_stop from generate)


In [ ]:
%%time
LLM_judge_score_fine_tuning = 0
for instruction, reference, gen_ending in zip(instructions_dataloader, reference_dataloader,generated_endings_dataloader):
  print(len(original_stories), len(reference))
  feedback, score = judge.absolute_grade(
      instructions=instruction,
      responses=gen_ending,
      rubric=score_rubric,
      reference_answers=reference
    )
  LLM_judge_score_fine_tuning += sum(score)
LLM_judge_score_fine_tuning = LLM_judge_score_fine_tuning/len(X_test)
LLM_judge_score_fine_tuning

# LLM as a judge for Baseline

In [47]:
import os
filepaths = [os.path.join(Output_DIR, 'generated_stories_beginning_baseline.txt'),
             os.path.join(Output_DIR, 'generated_stories_ending_baseline.txt'),
             os.path.join(Output_DIR, 'generated_text_baseline.txt')]

# Read the beginnings
with open(filepaths[0], 'r', encoding='utf-8') as f_beg:
    beginnings = f_beg.read().splitlines()  # This returns a list of strings, one per line

# Read the endings
with open(filepaths[1], 'r', encoding='utf-8') as f_end:
    endings = f_end.read().splitlines()

# Read the full texts
with open(filepaths[2], 'r', encoding='utf-8') as f_text:
    full_texts = f_text.read().splitlines()

# Now you have three arrays: beginnings, endings, and full_texts.
print(beginnings[:5])  # print first 5 items for a quick check
print(endings[:5])
print(full_texts[:5])

['Evan had been saving for years. He went to the dealership and bought a really fancy BMW. Evan was so proud of his new car. He showed it off around town.', "Serena was planning a surprise for her husband's birthday. She wanted to throw him a party, but his schedule was tough. He would always arrive home at widely different times. To get around it, she worked with his co-workers.", "Fred slapped another man's butt. He thought he knew the man. The man turned around. It was not the man he knew.", 'I used to lived in Phoenix Arizona. On my way home from work, i met my favorite actor. I was shocked by surprise when he actually spoke to me. We had lunch at the Olive Garden.', 'Tom thought he was really strong. He challenged all his friends to arm wrestle. Tom was able to beat them all. He would brag about it often.']
['    Evan had been saving for years. He went to the dealership and bought a really fancy BMW. E', ' ###### Instruction:Complete the story using a single sentence, based on the

In [49]:
generated_text_baseline = full_texts

In [53]:
generated_story_endings = []

for text in generated_text_baseline:
  ending = extract_story_ending_from_generated_text(text)
  generated_story_endings.append(ending)

len(generated_story_endings)

3535

In [54]:
instructions_dataloader = DataLoader(X_test, batch_size=batch_size)
reference_dataloader = DataLoader(y_test, batch_size=batch_size)
generated_endings_dataloader = DataLoader(generated_story_endings, batch_size=batch_size)

In [55]:
%%time
LLM_judge_score_baseline = 0
i = 0
for instruction, reference, gen_ending in zip(instructions_dataloader, reference_dataloader,generated_endings_dataloader):
  i+= 1
  feedback, score = judge.absolute_grade(
      instructions=instruction,
      responses=gen_ending,
      rubric=score_rubric,
      reference_answers=reference
    )
  LLM_judge_score_baseline += sum(score)
  print("iteration", i, "score:", score)
LLM_judge_score_baseline = LLM_judge_score_baseline/len(X_test)
LLM_judge_score_baseline

Processed prompts: 100%|██████████| 32/32 [00:08<00:00,  3.86it/s, est. speed input: 2389.16 toks/s, output: 629.75 toks/s]


Processed 32/32 instances.


Finalizing: 100%|██████████| 32/32 [00:00<00:00, 12186.10it/s]


iteration 1 score: [1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


Processed prompts: 100%|██████████| 32/32 [00:11<00:00,  2.76it/s, est. speed input: 1703.64 toks/s, output: 446.54 toks/s]


...

Processed prompts: 100%|██████████| 15/15 [00:05<00:00,  2.69it/s, est. speed input: 1665.92 toks/s, output: 427.61 toks/s]


Processed 15/15 instances.


Finalizing: 100%|██████████| 15/15 [00:00<00:00, 11635.76it/s]

iteration 111 score: [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1]
CPU times: user 16min 40s, sys: 2.42 s, total: 16min 42s
Wall time: 16min 36s


1.1796322489391797

In [56]:
print(LLM_judge_score_baseline)

1.1796322489391797


In [57]:
filepaths = [os.path.join(Output_DIR, 'generated_stories_beginning_lora_no_beam.txt'),
             os.path.join(Output_DIR, 'generated_stories_ending_lora_no_beam.txt'),
             os.path.join(Output_DIR, 'generated_text_lora_no_beam.txt')]

# Read the beginnings
with open(filepaths[0], 'r', encoding='utf-8') as f_beg:
    beginnings = f_beg.read().splitlines()  # This returns a list of strings, one per line

# Read the endings
with open(filepaths[1], 'r', encoding='utf-8') as f_end:
    endings = f_end.read().splitlines()

# Read the full texts
with open(filepaths[2], 'r', encoding='utf-8') as f_text:
    full_texts = f_text.read().splitlines()

# Now you have three arrays: beginnings, endings, and full_texts.
print(beginnings[:5])  # print first 5 items for a quick check
print(endings[:5])
print(full_texts[:5])

['Evan had been saving for years. He went to the dealership and bought a really fancy BMW. Evan was so proud of his new car. He showed it off around town.', "Serena was planning a surprise for her husband's birthday. She wanted to throw him a party, but his schedule was tough. He would always arrive home at widely different times. To get around it, she worked with his co-workers.", "Fred slapped another man's butt. He thought he knew the man. The man turned around. It was not the man he knew.", 'I used to lived in Phoenix Arizona. On my way home from work, i met my favorite actor. I was shocked by surprise when he actually spoke to me. We had lunch at the Olive Garden.', 'Tom thought he was really strong. He challenged all his friends to arm wrestle. Tom was able to beat them all. He would brag about it often.']
[' ###Evan was driving home one day. He was driving very fast. He hit a patch of ice. He', " her husband surprised her with a surprise party. He arrived home at 10pm. He was so

In [58]:
generated_text_lora = full_texts

In [59]:
generated_story_endings = []

for text in generated_text_lora:
  ending = extract_story_ending_from_generated_text(text)
  generated_story_endings.append(ending)

len(generated_story_endings)

3535

In [60]:
%%time
# Prepare the dataloaders
instructions_dataloader = DataLoader(X_test, batch_size=batch_size)
reference_dataloader = DataLoader(y_test, batch_size=batch_size)
generated_endings_dataloader = DataLoader(generated_story_endings, batch_size=batch_size)

LLM_judge_score_lora = 0
i = 0
for instruction, reference, gen_ending in zip(instructions_dataloader, reference_dataloader,generated_endings_dataloader):
  i+= 1
  feedback, score = judge.absolute_grade(
      instructions=instruction,
      responses=gen_ending,
      rubric=score_rubric,
      reference_answers=reference
    )
  LLM_judge_score_lora += sum(score)
  print("iteration", i, "score:", score)
LLM_judge_score_lora = LLM_judge_score_lora/len(X_test)
LLM_judge_score_lora

Processed prompts: 100%|██████████| 32/32 [00:11<00:00,  2.76it/s, est. speed input: 1709.43 toks/s, output: 480.09 toks/s]


Processed 32/32 instances.


Finalizing: 100%|██████████| 32/32 [00:00<00:00, 11506.02it/s]


iteration 1 score: [1, 1, 1, 3, 3, 2, 4, 4, 1, 2, 1, 3, 1, 3, 3, 1, 1, 3, 1, 4, 1, 3, 1, 1, 1, 1, 3, 1, 3, 1, 2, 2]


Processed prompts: 100%|██████████| 32/32 [00:08<00:00,  3.66it/s, est. speed input: 2264.57 toks/s, output: 644.94 toks/s]


...

Processed prompts: 100%|██████████| 15/15 [00:05<00:00,  2.58it/s, est. speed input: 1598.24 toks/s, output: 446.02 toks/s]


Processed 15/15 instances.


Finalizing: 100%|██████████| 15/15 [00:00<00:00, 11076.51it/s]

iteration 111 score: [1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 1, 3, 1, 1, 3]
CPU times: user 15min 7s, sys: 3.32 s, total: 15min 11s
Wall time: 15min 4s


1.8147100424328146

In [61]:
from google.colab import runtime
runtime.unassign()